# Funciones Empiricas PDF

Como calcular tos PDF, CDF and SF desde unos datos empiricos.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
xs = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 
      5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0]
ys = [0.2, 0.165, 0.167, 0.166, 0.154, 0.134, 0.117, 
      0.108, 0.092, 0.06, 0.031, 0.028, 0.048, 0.077, 
      0.103, 0.119, 0.119, 0.103, 0.074, 0.038, 0.003]

plt.scatter(xs, ys)
plt.xlabel("x")
plt.ylabel("Observos PDF");

: 

Suponiendo que esta es una función continua con solo observaciones escasas. Entonces, ¿cómo podemos llenar los espacios en blanco?

In [ ]:
x = np.linspace(min(xs), max(xs), 1000)
y1 = interp1d(xs, ys)(x)
y2 = interp1d(xs, ys, kind="Próximo")(x)
y3 = interp1d(xs, ys, kind="Cuadratico")(x)
y4 = interp1d(xs, ys, kind="Cubico")(x)

from scipy.interpolate import splev, splrep
y5 = splev(x, splrep(xs, ys))

plt.scatter(xs, ys, s=30, label="Datos", c="w")
plt.plot(x, y1, label="Lineal (defecto)")
plt.plot(x, y2, label="Próximo", alpha=0.2)
plt.plot(x, y3, label="Cuadratico", ls='-')
plt.plot(x, y4, label="Cúbico", ls='-')
plt.plot(x, y5, label="Spline", ls='-', alpha=1)
plt.legend();

Usando `interp1d` ahora podemos encontrar un valor de probabilidad para cualquier valor de `x`.

* `scipy.integrate.trapz` para baja precisión pero alta velocidad. Escalas de precisión como `O(h)`
* `scipy.integrate.simps` para una precisión media y una velocidad bastante alta. Escalas de precisión como `O(h^2)`
* `scipy.integrate.quad` para alta precisión y baja velocidad. Precisión arbitraria.


In [ ]:
from scipy.integrate import simps

def obtener_prob(xs, ys, a, b, resolution=1000):
    x_norm = np.linspace(min(xs), max(xs), resolution)
    y_norm = interp1d(xs, ys, kind="cuadratico")(x_norm)
    normalisation = simps(y_norm, x=x_norm)
    x_vals = np.linspace(a, b, resolution)
    y_vals = interp1d(xs, ys, kind="cuadratico")(x_vals)
    return simps(y_vals, x=x_vals) / normalisation

def obtener_cdf(xs, ys, v):
    return obtener_prob(xs, ys, min(xs), v)

def obtener_sf(xs, ys, v):
    return 1 - obtener_cdf(xs, ys, v)

print(obtener_prob(xs, ys, 0, 10))

In [ ]:
v1, v2 = 6, 9.3
area = obtener_prob(xs, ys, v1, v2)

plt.scatter(xs, ys, s=30, label="Datos", color="w")
plt.plot(x, y3, linestyle="-", label="Interpolacion")
plt.fill_between(x, 0, y3, where=(x>=v1)&(x<=v2), alpha=0.2)
plt.annotate(f"p = {area:.3f}", (7, 0.05))
plt.legend();

In [ ]:
x_new = np.linspace(min(xs), max(xs), 100)
cdf_new = [obtener_cdf(xs, ys, i) for i in x_new]
cheap_cdf = y3.cumsum() / y3.sum()

plt.plot(x_new, cdf_new, label="Interpolato CDF")
plt.plot(x, cheap_cdf, label="Super trampa CDF para casos específicos")
plt.ylabel("CDF")
plt.xlabel("x")
plt.legend()
plt.show();